In [1]:
import numpy as np

from Constant import *
from Model_Structure import *
from dataset import *

from keras.models import Model
from keras.optimizer_v1 import Adam
from keras.losses import categorical_crossentropy, sparse_categorical_crossentropy
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau


In [8]:
# 加载数据
X_train, Y_train, X_test, Y_test = load_radio2016_regress(mod_snr=('8PSK', 18))

In [9]:
print(np.shape(X_train), np.shape(Y_train), np.shape(X_test), np.shape(Y_test))

(688, 2, 127) (688, 2) (312, 2, 127) (312, 2)


In [10]:
inshape = (2, 127)
input = Input(shape=inshape)
output = LSTM(units=256, activation="tanh", return_sequences=True)(input)
output = LSTM(units=128, activation="tanh", return_sequences=True)(output)

output = Flatten()(output)

output = Dense(1024, activation=None)(output)
output = BatchNormalization()(output)
output = Activation(activation="relu")(output)
output = Dropout(0.5)(output)

output = Dense(128, activation=None)(output)
output = BatchNormalization()(output)
output = Activation(activation="relu")(output)
output = Dropout(0.5)(output)

output = Dense(16, activation=None)(output)
output = BatchNormalization()(output)
output = Activation(activation="relu")(output)
output = Dropout(0.5)(output)

output1 = Dense(2, activation='sigmoid')(output)

In [11]:
model = Model(input, output1)
optimizer = Adam(lr=1e-4)
model.compile(optimizer=optimizer, loss="mse", metrics=["mse"])
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 2, 127)]          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 2, 256)            393216    
_________________________________________________________________
lstm_3 (LSTM)                (None, 2, 128)            197120    
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              263168    
_________________________________________________________________
batch_normalization_3 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_3 (Activation)    (None, 1024)              0   

In [12]:
batch_size = 8
epochs = 10
# 加载模型
# model = radio2016_model(lr=1e-4)
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), cooldown=0, patience=2, min_lr=1e-5)
model_checkpoint = ModelCheckpoint("Weights/signal.h5", monitor="val_loss",
                                   save_best_only=True, save_weights_only=False, verbose=1)
callbacks = [model_checkpoint, lr_reducer]

model.fit(x=X_train, y=Y_train, batch_size=batch_size, epochs=epochs, shuffle=True,
          callbacks=callbacks, validation_data=(X_test, Y_test), verbose=1)

Train on 688 samples, validate on 312 samples
Epoch 1/10
688/688 [==============================] - ETA: 0s - loss: 0.0834 - mse: 0.0834

/public/liujiawei/anaconda3/envs/ZHB_env/lib/python3.6/site-packages/keras/engine/training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '



Epoch 00001: val_loss improved from inf to 0.01110, saving model to Weights/signal.h5
688/688 [==============================] - 5s 8ms/sample - loss: 0.0834 - mse: 0.0834 - val_loss: 0.0111 - val_mse: 0.0111 - lr: 1.0000e-04
Epoch 2/10
664/688 [===========================>..] - ETA: 0s - loss: 0.0824 - mse: 0.0824
Epoch 00002: val_loss did not improve from 0.01110
688/688 [==============================] - 1s 2ms/sample - loss: 0.0824 - mse: 0.0824 - val_loss: 0.0125 - val_mse: 0.0125 - lr: 1.0000e-04
Epoch 3/10
688/688 [==============================] - ETA: 0s - loss: 0.0734 - mse: 0.0734
Epoch 00003: val_loss did not improve from 0.01110
688/688 [==============================] - 2s 2ms/sample - loss: 0.0734 - mse: 0.0734 - val_loss: 0.0138 - val_mse: 0.0138 - lr: 1.0000e-04
Epoch 4/10
688/688 [==============================] - ETA: 0s - loss: 0.0715 - mse: 0.0715
Epoch 00004: val_loss did not improve from 0.01110
688/688 [==============================] - 1s 2ms/sample - loss: 0.

In [13]:
model.evaluate(X_train, Y_train)

[0.028607895597815514, 0.028607896]

In [14]:
model.evaluate(X_test, Y_test)

[0.04204105595365549, 0.042041056]